In [ ]:
# Homomorphic encryption
# in simple words - a kind of encryption that allows to do computation over the ciphertexts
# note, that the operation over the ciphertexts may or may not be the same as over the plaintexts

In [ ]:
# RSA
# E(m1)*E(m2) = m1^e*m2^e = (m1*m2)^e = E(m1*m2)

# setup
size = 20
p = random_prime(2**21, lbound = 2**20)
q = random_prime(2**21, lbound = 2**20)
N = p*q
e = 65537
dp = (1/e) % (p-1)
dq = (1/e) % (q-1)
d = CRT([dp,dq], [p-1,q-1])

R = Integers(N)

print ("Our RSA modulus is {}, the factors are {} and {}".format(N,p,q))
print ("Our public exponent is {} and our private exponent is {}\n".format(e,d))

# message generation
m1 = R.random_element()
m2 = R.random_element()

# encryption
c1 = m1^e
c2 = m2^e

# computation

c3 = c1 * c2

# decryption
m3 = c3^d

print ("We have chosen {} and {} as our plaintexts, the corresponding ciphertexts are {} and {}\n".format(m1,m2,c1,c2))

print ("The product of the plaintexts is {}, the decryption of the product of the ciphertexts is {}".format(m1*m2, m3))
print ("That these two are equal is: {}".format(m3==m1*m2))

Our RSA modulus is 2422161987647, the factors are 1324681 and 1828487
Our public exponent is 65537 and our private exponent is 60298032233

We have chosen 577760427878 and 1768519523233 as our plaintexts, the corresponding ciphertexts are 770081663997 and 1634756781984

The product of the plaintexts is 2080817990534, the decryption of the product of the ciphertexts is 2080817990534
That these two are equal is: True


In [ ]:
# Caesar cipher
# E(m1)||E(m2) = E(m1||m2)
# Actually works for ciphers in ECB mode, if you concatenate with respect to block size

# meaning the rotation of the alphabet
key = 9

m1 = "river" 
m2 = "arena"

print ("Our secret alphabet shift will be by {}, our plaintexts will be {} and {}".format(key,m1,m2))


c1 = "".join([chr((ord(c)-97 + key) % 26 + 97) for c in m1])
c2 = "".join([chr((ord(c)-97 + key) % 26 + 97) for c in m2])

print ("The corresponding ciphertexts are {} and {}".format(c1,c2))

c3 = "".join([chr((ord(c)-97 + key) % 26 + 97) for c in m1+m2])

print ("Their concatenation is {}".format(c1+c2))
print ("The encryption of the concatenation of the ciphertexts is {}".format(c3))
print ("Are these two the same? ({})".format(c1+c2==c3))

Our secret alphabet shift will be by 9, our plaintexts will be river and arena
The corresponding ciphertexts are river and arena
Their concatenation is arenajanwj
The encryption of the concatenation of the ciphertexts is arenajanwj
Are these two the same? (True)


In [ ]:
# Pailier scheme
# E(m1)*E(m2) = (g^m1*r_1^n)*(g^m2*r_2^n) = g^(m1+m2) (r_1*r_2)^n = E(m1+m2)
# See, that the operation on plaintexts, and the corresponding operation on ciphertexts do not need to be equal


# setup
# we are using primes of the same size - setup is a bit easier
size = 20
p = random_prime(2**21, lbound = 2**20)
q = random_prime(2**21, lbound = 2**20)
l = lcm(p-1,q-1)
N = p*q

R = Integers(N^2)

g = R(N+1)

up = (1/l) % p
uq = (1/l) % q
u = CRT([up,uq], [p,q])

# choosing plaintexts
m1 = R(randint(1,N-1))
m2 = R(randint(1,N-1))

# choosing random "noise"
r1 = R(randint(1,N-1))
while gcd(r1,N)>1:
    r1 = R(randint(1,N-1))

r2 = R(randint(1,N-1))
while gcd(r2,N)>1:
    r2 = R(randint(1,N-1))

# encryption    
c1 = g^m1 * r1^N
c2 = g^m2 * r2^N

# computation
m3 = (m1 + m2) %N
c3 = c1*c2

# decryption
c = ZZ(c3)
l = ZZ(l)
u = ZZ(u)

t = ZZ(pow(c3,l,N^2))
dec = ((t-1)/N * u)%N

print ("We have chose our plaintexts to be {} and {}, their sum modulo N is {}".format(m1,m2,m3))
print ("We have computed the correspondning ciphertexts to be {} and {}, their product modulo N^2 is {}".format(c1,c2,c3))
print ("The decrypted product gives {}. Is it the same as the sum of the plaintexts: {}".format(dec, dec==m3))


We have chose our plaintexts to be 1494599460061 and 1946860094245, their sum modulo N is 713771782333
We have computed the correspondning ciphertexts to be 2630221854169506041968538 and 7244991171959294911631346, their product modulo N^2 is 4878774583414941475838989
The decrypted product gives 713771782333. Is it the same as the sum of the plaintexts: True


In [ ]:
# Now for the more interesting stuff: Fully Homomorphic Encryption (FHE)
# FHE means, that the arbitrary computation can be done on ciphertexts
# Needs just 2 operations: + and * 
# With these, arbitrary polynomial functions may be computed
# All functions over a finite field can be written in a polynomial form
# Computers are finite

# It was a longstanding open question whether this is even possible while maintaining security of cryptosystem
# In his seminal work Gentry provided a method, which most of current scheme follows
# https://crypto.stanford.edu/craig/craig-thesis.pdf

# Idea: Let us have an encryption method, which adds noise to the ciphertexts and is "homomorphic"
# (this gives so called Somewhat Homomorphic, or Leveled Homomorphic Encryption -- can evaluate circuits up to certain depth)
# If we decrypt, we rid ourselves of the noise, but we will remove the encryption
# We can however compose decryption and encryption into a "recryption" -- this function will be called on a too noisy ciphertext
# This will allow us to keep noise manageable (and is often called "bootstraping")

# if you want to play with some C++ library, you can try HElib https://github.com/homenc/HElib
# and/or you can take a look at wikipedia, and search for homomorphic encryption - there are links

In [ ]:
import random
# "2nd generation scheme" -- over the integers (although, ther is a connection to lattices) for the ease of exposition
# more details can be found here: https://eprint.iacr.org/2009/616.pdf
# we will be able to encrypt just bits

#note that this is just "somewhat homomorphic scheme" we have no way how to get rid of noise

# setup
l = 10
# these parameters should depend on lambda in a specified way for security
# but these are just toy parameters...
rho = l
rho2= l
eta = l^2
gamma = l^5


tau = gamma + l


# key genereration

# need an odd number p of a given length
# this is a private key
p = randint(2^(eta-1),2^eta-1)
while p%2 == 0:
    p = randint(-2^l+1,2^l-1)

# and we need a list of integers sampled from a given distribution 
# (can be seen as "encryptions of 0")
# this is a public key

x_list = []
for i in range(tau):
    x_list.append(p*randint(0,floor(2^gamma/p))+randint(-2^rho+1,2^rho-1))
x_list = sorted(x_list)

if x_list[0]%2==0:
    x_list[0]+=1
#if (x_list[0]%p)%2 == 1:
#    print("problem")

#print(x_list)

# now, let us choose our mesagges
m1 = m2 = m3 = 0
m4 = m5 = m6 = 1

m_list = [m1, m2, m3, m4, m5, m6]
print(m_list[:3])
print(m_list[3::])

# to encrypt the messages choose a random subset from x_list and a random integer r
# c = m+2*r+2*sum(x_i for x_i in x_list if x_i in subset)

c_list = []

for i in range(len(m_list)):
    #note that the 1 here is completely arbitrary
    subset = random.sample(x_list,1)
    c = m_list[i]+2*randint(-2^rho2+1,2^rho2-1)#+2*sum(subset)
    c %= x_list[0]
    c_list.append(c)
    
print(c_list[5])

#print(c_list[:3])
#print(c_list[3::])

#this is (or should be) decryption
print ([((c % p) % 2) for c in c_list])

#should get 0
print ( (sum(c_list[:3])%p %2))
print ( (product(c_list[:3])%p %2))
#should get 1
print ( (sum(c_list[3::])%p %2))
print ( (product(c_list[3::])%p %2))    

print ([((c % p) % 2) for c in  [c_list[0]+c_list[3],c_list[1]+c_list[4],c_list[2]+c_list[5]] ])

ValueError: empty range for randrange() (0, -1834228958681090595554980279350784540877775474881302275904805007213081271773871230611410183179611039220444521370990381119038711414448475316229278665510723622033250149432688931911643391602339149951460649859855759109549676184061360360289342230575358623042233023631015625393540013732420865059985593478439398693115737248464017821944126129145372682676469250126647754950195572524431284321724278895569816087141463299384907985055527328509029875471692965889495164008881651454668556309386793237756344524915649786960300946011844843869543785836794229133697633096229398274420326505513632506652435749765309468385509819326654110421101252750419879281975715478819535039952193949191097468459495303417476800254432282294299575169017920919608375745307641075800261532323311527391943065228873338195537414941917542473528858703816513646865689626377755319942655109777989763729000513610349512520640992788229392371296677210378444221482097454573918206202107045594469472311636013576100

In [ ]:
# to make the scheme fully homomorphic, some changes must be made - public key must contain a "hint" about private key
# not so much to allow break, but enough, to allow a simpler decryption
# the idea is as follows: add 3 more parameters
# and we give some information about 1/p

In [ ]:
# The lattice connection:
# the more efficient/successful FHE schemes are based on lattices
# the very first one too
# and the lattice attacks are the some of the most powerful attacks against these schemes
# for this scheme the security is based on "approximate gcd assumption"
# all ciphertexts are "almost" multiples of p
# uncover p -> break the scheme
# lattice attacks are the reason for high security parameters and thus the unwieldiness of these schemes